## CLASS torch.nn.Module

Base class for all neural network modules. Your models should alse subclass this class.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(conv1(x))
        return F.relu(conv2(x))

## CLASS torch.nn.Sequential(\*args)

A sequential container.Modules will be added to it in the order they are passed in(传入) the constructor. Alternatively, an ordered dict of modules can also be passed in.

In [3]:
from collections import OrderedDict
# Example of using Sequential
model = nn.Sequential(
    nn.Conv2d(1, 20, 5),
    nn.ReLU(),
    nn.Conv2d(20, 64, 5),
    nn.ReLU()
    )
# Example of using Sequential with OrderedDict
model = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 20, 5)),
    ('ReLU1', nn.ReLU()),
    ('conv2', nn.Conv2d(20, 64, 5)),
    ('ReLU2', nn.ReLU())
    ]))

nn.Sequential里面的模块按照顺序进行排列的，所以必须确保前一个模块的输出大小和下一个模块的输入大小时一致的。

## CLASS torch.nn.ModuleList(modules=None)

nn.ModuleList，它是一个储存不同 module，并自动将每个 module 的 parameters 添加到网络之中的容器。你可以把任意 nn.Module 的子类 (比如 nn.Conv2d, nn.Linear 之类的) 加到这个 list 里面，方法和 Python 自带的 list 一样，无非是 extend，append 等操作。但不同于一般的 list，加入到 nn.ModuleList 里面的 module 是会自动注册到整个网络上的，同时 module 的 parameters 也会自动添加到整个网络中。若使用python的list，则会出问题。下面看一个例子：

In [36]:
class net_modlist(nn.Module):
    def __init__(self):
        super(net_modlist, self).__init__()
        self.modlist = nn.ModuleList([
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        ])

    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

net_modlist = net_modlist()
print(net_modlist)

net_modlist(
  (modlist): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)


In [37]:
for param in net_modlist.parameters():
    print(type(param.data), param.size())

<class 'torch.Tensor'> torch.Size([20, 1, 5, 5])
<class 'torch.Tensor'> torch.Size([20])
<class 'torch.Tensor'> torch.Size([64, 20, 5, 5])
<class 'torch.Tensor'> torch.Size([64])


In [38]:
class net_modlist(nn.Module):
    def __init__(self):
        super(net_modlist, self).__init__()
        self.modlist = nn.ModuleList()
        self.modlist.append(nn.Conv2d(1, 20, 5))
        self.modlist.append(nn.ReLU())
        self.modlist.append(nn.Conv2d(20, 64, 5))
        self.modlist.append(nn.ReLU())
    
    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x
    
net_modlist = net_modlist()
print(net_modlist)

net_modlist(
  (modlist): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)


In [32]:
for param in net_modlist.parameters():
    print(type(param.data), param.size())

<class 'torch.Tensor'> torch.Size([20, 1, 5, 5])
<class 'torch.Tensor'> torch.Size([20])
<class 'torch.Tensor'> torch.Size([64, 20, 5, 5])
<class 'torch.Tensor'> torch.Size([64])


接下来看看另一个作为对比的网络，它使用 Python 自带的 list：

In [8]:
class net_modlist(nn.Module):
    def __init__(self):
        super(net_modlist, self).__init__()
        self.modlist = [
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        ]

    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

net_modlist = net_modlist()
print(net_modlist)

net_modlist()


In [10]:
for param in net_modlist.parameters():
    print(type(param.data), param.size())

显然，使用 Python 的 list 添加的卷积层和它们的 parameters 并没有自动注册到我们的网络中。当然，我们还是可以使用 forward 来计算输出结果。但是如果用其实例化的网络进行训练的时候，因为这些层的parameters不在整个网络之中，所以其网络参数也不会被更新，也就是无法训练。

In [4]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linears = nn.ModuleList([nn.linear for i in range(10)])

    # ModuleList can act as an iterable, or be indexed using ints
    def forward(self, x):
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

## nn.Sequential 与 nn.ModuleList的区别

* 不同点1：nn.Sequential内部实现了forward函数，因此可以不用写forward函数。而nn.ModuleList则没有实现内部forward函数。

In [11]:
# 内部已经实现了forward函数，所以不用写def forward()
seq = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
print(seq)

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)


In [13]:
input = torch.randn(16, 1, 20, 20)
seq(input).size()

torch.Size([16, 64, 12, 12])

但如果是继承nn.Module类的话，就要写forward函数

In [14]:
class net1(nn.Module):
    def __init__(self):
        super(net1, self).__init__()
        self.seq = nn.Sequential(
                        nn.Conv2d(1, 20, 5),
                        nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        )
    
    def forward(self, x):
        return self.seq(x)
    
    # 注意：按照下面这种利用for循环的方式也可以得到同样的结果
    # def forward(self, x):
    #     for s in self.seq:
    #         x = s(x)
    #     return x
    
input = torch.randn(16, 1, 20, 20)
net1 = net1()
print(net1(input).shape)

torch.Size([16, 64, 12, 12])


而对于nn.ModuleList:

In [15]:
#例1：若按照下面这么写，则会产生错误
modlist = nn.ModuleList([
         nn.Conv2d(1, 20, 5),
         nn.ReLU(),
         nn.Conv2d(20, 64, 5),
         nn.ReLU()
         ])
print(modlist)

ModuleList(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)


In [16]:
input = torch.randn(16, 1, 20, 20)
print(modlist(input))

TypeError: forward() takes 1 positional argument but 2 were given

In [17]:
# 更改：加写了forward函数
class net2(nn.Module):
    def __init__(self):
        super(net2, self).__init__()
        self.modlist = nn.ModuleList([
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        ])
        
    #这里若按照这种写法则会报NotImplementedError错
    #def forward(self, x):
    #    return self.modlist(x)

    #注意：只能按照下面利用for循环的方式
    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x
    
input = torch.randn(16, 1, 20, 20)
net2 = net2()
print(net2(input).shape)

torch.Size([16, 64, 12, 12])


如果完全直接用 `nn.Sequential`，确实是可以的，但这么做的代价就是失去了部分灵活性，不能自己去定制 forward 函数里面的内容了。

一般情况下 `nn.Sequential` 的用法是来组成卷积块 (block)，然后像拼积木一样把不同的 block 拼成整个网络，让代码更简洁，更加结构化。

* 不同点2：nn.Sequential可以使用OrderedDict对每层进行命名，上面已经阐述过了；

* 不同点3：nn.Sequential里面的模块按照顺序进行排列的，所以必须确保前一个模块的输出大小和下一个模块的输入大小是一致的。而nn.ModuleList 并没有定义一个网络，它只是将不同的模块储存在一起，这些模块之间并没有什么先后顺序可言。

见下面代码：

In [18]:
class net3(nn.Module):
    def __init__(self):
        super(net3, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,20), nn.Linear(20,30), nn.Linear(5,10)])
    def forward(self, x):
        x = self.linears[2](x)
        x = self.linears[0](x)
        x = self.linears[1](x)

        return x

net3 = net3()
print(net3)

net3(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=30, bias=True)
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
)


In [19]:
input = torch.randn(32, 5)
print(net3(input).shape)

torch.Size([32, 30])


根据 net3 的结果，可以看出来这个 `ModuleList` 里面的顺序不能决定什么，网络的执行顺序是根据 `forward` 函数来决定的。若将`forward`函数中几行代码互换，使输入输出之间的大小不一致，则程序会报错。此外，为了使代码具有更高的可读性，最好把`ModuleList`和`forward`中的顺序保持一致。

* 不同点4：有时候网络中有很多相似或者重复的层，我们一般会考虑用for循环来创建他们，而不是一行一行地写

比如：`layer=[nn.Linear(10, 10) for i in range(5)]`

那么这里我们使用`ModuleList`

In [20]:
class net4(nn.Module):
    def __init__(self):
        super(net4, self).__init__()
        layers = [nn.Linear(10, 10) for i in range(5)]
        self.linears = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.linears:
            x = layer(x)
        return x

net = net4()
print(net)

net4(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)
